In [ ]:
import bs4
import warnings
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from IPython.display import (display, HTML, display_html,
                             Markdown, Latex)
from ipywidgets import (Box, HBox, VBox, Layout,
                        Button, HTML as HTML_widget)
from astropy.coordinates import SkyCoord
from astropy.table import Table
from functools import partial
from bokeh.plotting import show, output_notebook
from snII_cosmo_tools import *

In [ ]:
rsc = GladeRedshiftCatalogue('redshift_data/glade_v2.3.hdf')

In [ ]:
tns_request = TNSDownloader(number_of_days=2)
#tns_request = TNSDownloader(number_of_days=150, obj_type='SN II')
targets = insert_max_alt_in_frame(tns_request.result_compact)

# Classification targets

In [ ]:
targets, names = insert_tns_links_into_df(targets)
targets = insert_gal_lat_in_frame(targets)
targets = insert_survey_links_into_df(targets)
targets.index = names

In [ ]:
tf = InteractiveTargetFilter(targets)
targets_filtered = tf.interactive

display(targets_filtered)

In [ ]:
highlight_z = partial(highlight_general_traffic_light, levels=[0.03, 0.04, 0.12])
highlight_abs_mag = partial(highlight_general_traffic_light, levels=[-18.0, -17.5, 1e4])

In [ ]:
%matplotlib widget
plt.ioff()
output_notebook()
aladins = []
tables = []

layout = Layout(width='100%')
layout.margin = '100px px 0px 0px'

for i in range(len(targets_filtered.result)):
    widgets = []

    df_object = pd.DataFrame(targets_filtered.result.iloc[i].values[np.newaxis,:],
                             columns=targets.columns)
    
    row = targets_filtered.result.iloc[i]
    target_vis = TargetVisualizer.from_row(row)
    
    # Visibility
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        vis = InteractiveVisibility(skycoord=target_vis.coords)
        vis_fig = vis.fig
    
    # Redshift
    m = rsc.find_host_z_from_row(row, max_dist_kpc=40.)
    checks = []
    if len(m) > 0 or np.isfinite(row['Host Redshift']):
        for brightness_category in ['bright', 'medium', 'faint']:
            template = 'data/template_obs/ob_classification_{}.obx'.format(brightness_category)
            obgen = OBGenerator.from_row(row, template_ob=template)
            if obgen.magnitude.brightness_category == brightness_category:
                button_style = 'success'
            else:
                button_style = 'danger'
            check = Button(description="OB {}".format(brightness_category),
                           button_style=button_style)
            check.on_click(obgen)
            checks.append(check)
        checks.append(HTML_widget("&emsp; "*8))
        checks.append(vis.interactive)
        display(target_vis.html_label)
        

        widgets.append(HBox(checks))
        a = TargetArchiver(row)
        widgets.append(HBox([a.button, VBox([a.destination, a.text])]))
        box = HBox([target_vis.aladin, vis_fig.canvas], layout=layout)
        widgets.append(box)

        vbox = VBox(widgets, layout=Layout(width='100%'))

        display(vbox)

        checks.append(check)

        sdss_frame = pd.DataFrame([target_vis.sdss_html],
                                  columns=['SDSS Skyserver'])
        df_object_sdss = pd.concat([df_object, sdss_frame], axis=1)
        display(HTML(get_styled_html_table(df_object_sdss)))
        
        if len(m) > 0:
            redshift_table = m[rsc.comp_repr]
            caption = 'Possible glade hosts'
            table = rsc.generate_aladin_table(m)
            aladins.append(target_vis.aladin)
            tables.append(table)
        else:
            redshift_table = rsc.get_tns_host_from_row(row)
            caption = 'TNS host'
        display(HTML(redshift_table.style.apply(
                highlight_z, subset=['z']).apply(highlight_abs_mag, subset=['abs_mag']
                                            ).hide_index().set_caption(
                                                    caption).set_properties(**{'width': '100%'}).render()))
            
        
        if tf.obj_type.value != 'unclassified':
            try:
                spec = TNSSpectrum(row.name)
                class_fig = spec.plot()
                display(class_fig.canvas)
            except TNSDownloadError:
                print('No classification spectrum found')
        internal_name = row.loc['Disc. Internal Name']
        if type(internal_name) is str and 'ZTF' in internal_name:
            try:
                ztf_lc = ZtfLightCurveDownloader(
                    bs4.BeautifulSoup(internal_name).text
                )
                lc_fig = ztf_lc.plot()
                display(HTML('<br/>'))
                show(lc_fig)
            except:
                print('Failed plotting ZTF light curve!')

    else:
        continue
    display(HTML('<br/><br/><br/>'))

# Add host galaxies to aladin widget
for aladin, table in zip(aladins, tables):
    aladin.add_table(table)

## TODO:

* highlight spectroscopic redshifts
* add 2df redshift data, SDSS?
* check separation calculation (specifically ordering)
* option to archive targets